## Variables

In [20]:
# stagging path
source_path = "source_files"

# path to sources
tables = {
    "products": {
        "path": "products (1).csv",
        "PK": "ProductID",  # Chave primária
        "FK": []  # Lista de chaves estrangeiras associadas (vazia se não houver)
    },
    "sales_order_detail": {
        "path": "sales_order_detail.csv",
        "PK": "SalesOrderDetailID",  # Chave primária
        "FK": ["order_id"]  # Supondo que o order_id seja uma chave estrangeira
    },
    "sales_order_header": {
        "path": "sales_order_header (1).csv",
        "PK": "SalesOrderID",  # Chave primária
        "FK": []  # Lista de chaves estrangeiras associadas (vazia se não houver)
    }
}

## Spark Session

In [28]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# create spark Session
spark = SparkSession.builder \
    .appName("Pipeline") \
    .getOrCreate()

# show spark version
print(spark.version)

# Configurar o logger para ERROR - Spark Run A Lot of Error Running Locally 
spark.sparkContext.setLogLevel("ERROR")

3.5.5


## Raw Function

In [31]:
def run_raw(tables={}):    

    # for each table in the parameter list
    for table_name, table_params in tables.items():
        
        # create table path
        table_path = f"{source_path}/{table_params['path']}"

        # load csv using header and loading schema from data-sample
        stagging_table = spark.read.csv(table_path, header=True, inferSchema=True)
        
        # grant unique PK
        if stagging_table.groupBy(table_params['PK']).count().filter(col("count") > 1).count() > 0:
            raise ValueError(f"column '{table_params['PK']}' has duplicates.")

        # save raw table
        stagging_table.write \
            .mode("overwrite") \
            .parquet(f"data/raw_{table_name}", compression="none")

In [32]:
run_raw(tables)

ValueError: column 'ProductID' has duplicates.

In [4]:
df = spark.read.csv("source_files/products (1).csv", header=True, inferSchema=True)

In [11]:
df.write.format("parquet").mode("overwrite").save("data/table1")

In [8]:
df.write \
    .mode("overwrite") \
    .parquet("/data/table", compression="none")

25/03/20 23:05:57 ERROR FileOutputCommitter: Mkdirs failed to create file:/data/table/_temporary/0
25/03/20 23:05:57 ERROR Executor: Exception in task 0.0 in stage 3.0 (TID 3)
java.io.IOException: Mkdirs failed to create file:/data/table/_temporary/0/_temporary/attempt_202503202305578550225663484862037_0003_m_000000_3 (exists=false, cwd=file:/Users/amadoroque/Downloads/upstart13)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:515)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:500)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1195)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1175)
	at org.apache.parquet.hadoop.util.HadoopOutputFile.create(HadoopOutputFile.java:74)
	at org.apache.parquet.hadoop.ParquetFileWriter.<init>(ParquetFileWriter.java:347)
	at org.apache.parquet.hadoop.ParquetFileWriter.<init>(ParquetFileWriter.java:314)
	at org.apache.parquet.hadoop.ParquetOutputFormat.getRecordWriter(ParquetOut

Py4JJavaError: An error occurred while calling o48.parquet.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 1 times, most recent failure: Lost task 0.0 in stage 3.0 (TID 3) (192.168.15.10 executor driver): java.io.IOException: Mkdirs failed to create file:/data/table/_temporary/0/_temporary/attempt_202503202305578550225663484862037_0003_m_000000_3 (exists=false, cwd=file:/Users/amadoroque/Downloads/upstart13)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:515)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:500)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1195)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1175)
	at org.apache.parquet.hadoop.util.HadoopOutputFile.create(HadoopOutputFile.java:74)
	at org.apache.parquet.hadoop.ParquetFileWriter.<init>(ParquetFileWriter.java:347)
	at org.apache.parquet.hadoop.ParquetFileWriter.<init>(ParquetFileWriter.java:314